In [32]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from ovejero import bnn_alexnet, model_trainer, data_tools

def NOTIMPLEMENTED():
    raise NotImplementedError('Must specify config path')

# Testing the Performance of a Model That Has Been Fit

__Author:__ Sebastian Wagner-Carena

__Created:__ 10/24/2019

__Last Run:__ 11/7/2019

__Goals:__ Learn how to test the performance of a trained model on the validation set.

__Before running this notebook:__ Run the Train_Toy_Model notebook to understand how to train a model. Then train a model with whatever configuration you want. You will have to add the path to the config file in this notebook.

In [33]:
# First specify the config path
config_path = '../configs/t1_local.json' #NOTIMPLEMENTED()

# Check that the config has what you need
cfg = model_trainer.load_config(config_path)

# Get the model (ignore the loss function since we won't use it)
model, _ = model_trainer.model_loss_builder(cfg)

# Load the validation data we're going to use to test the performance of our model
tf_record_path_v = cfg['validation_params']['root_path']+cfg['validation_params']['tf_record_path']
final_params = cfg['training_params']['final_params']
batch_size = 128

In [40]:
tf_dataset_v = data_tools.build_tf_dataset(tf_record_path_v,final_params,batch_size,1)
image_batch, output_batch = tf_dataset_v.take(1)

NotFoundError: /Users/sebwagner/Documents/Grad_School/Rotations/Phil/ovejero/datasets/train/tf_record_val; No such file or directory [Op:IteratorGetNextSync]